In [1]:
# pip install -U spacy
# python -m spacy download en_core_web_sm
# %pip install openai

import pandas as pd
import numpy as np
from openai import OpenAI
import json
from collections import OrderedDict
import time

In [2]:
philly_reviews = pd.read_csv('philadelphia_restaurant_reviews_sampled(in).csv')
# for testing purposes: 
philly_reviews = philly_reviews.head(1)

In [4]:
client = OpenAI()

annotated_reviews = []

start = time.time()
for idx, review in philly_reviews.iterrows():
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a restaurant review annotator. Categorize the following text so I can evaluate the restaurant accurately. Then for each category give the restaurant a score from 1-5, 1 being bad and 5 being excellent for their performance in each category. Include every single dish mentioned in the review. Return the output in three column JSON format with category and score."},
            {"role": "user", "content": review['text']},
        ],
        max_tokens=256
    )

    enriched = response.choices[0].message.content.removeprefix('```json').removesuffix('```').strip()
    
    try:
        enriched = json.loads(enriched)
    except:
        print('Error parsing JSON')
        print(enriched)
        enriched = pd.NA
        continue
    
    annotated_reviews.append(enriched)

tot_time = time.time() - start
print(tot_time)
  
philly_reviews['review_scores'] = annotated_reviews
